In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import uuid
import logging
import getpass
import numpy as np
import cv2
import os
import gc
import time
import imageio
import random
import shutil
import copy
import glob
import json
import shutil
import pickle
import traceback
from collections import Counter
from pprint import pprint as print

In [3]:
import cv2
import requests
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from tqdm import tqdm, tqdm_notebook
from PIL import Image, ImageDraw
from shapely.geometry import Polygon
import imgaug
import imgaug as ia
import imgaug.augmenters as iaa

import torch
from torch.utils.data import DataLoader

In [4]:
def imshow(img, figsize=(16, 16)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(img)

In [ ]:
img_dir = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/V1/images"

In [ ]:
img_fps = glob.glob(os.path.join(img_dir, "*.jpg"))
print(len(img_fps))

In [ ]:
front_img_fps = sorted([i for i in img_fps if "_front." in i])
back_img_fps = sorted([i for i in img_fps if "_back." in i])
print(len(front_img_fps))
print(len(back_img_fps))

In [ ]:
save_image_fd = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/CVAT/cropper"
assert os.path.exists(save_image_fd)

In [ ]:
def remove_all_content(fd):
    shutil.rmtree(fd)
    os.mkdir(fd)
    
# remove_all_content(save_image_fd)

In [ ]:
def create_batches_image(img_fps, suffix=""):
    # create batch image first
    batch_index = 0
    batch_dir_fmt = "{}{}"
    max_img_per_batch = 200

    for index, img_fp in tqdm_notebook(enumerate(img_fps)):

        saved_batch_dir = os.path.join(save_image_fd, batch_dir_fmt.format(batch_index, suffix))
        if not os.path.exists(saved_batch_dir):
            os.mkdir(saved_batch_dir)

        img_fn = img_fp.strip("/").split("/")[-1]
        new_img_fn = os.path.join(saved_batch_dir, img_fn)
        shutil.copy(img_fp, new_img_fn)

        if ((index + 1) % max_img_per_batch == 0 and index > 0) or index == len(img_fps) - 1:
            shutil.make_archive(
                saved_batch_dir,
                "zip", saved_batch_dir
            )
            batch_index += 1

    print(batch_index)
    shutil.make_archive(
        saved_batch_dir,
        "zip", saved_batch_dir
    )
    print(">>> Completed!")

In [ ]:
create_batches_image(front_img_fps, suffix="_front")
create_batches_image(back_img_fps, suffix="_back")

### Upload by CVAT API

In [12]:
LABELS = [
    {"name":"top_left", "attributes": []},
    {"name":"top_right", "attributes": []},
    {"name":"bottom_left", "attributes": []},
    {"name":"bottom_right", "attributes": []},
    {"name":"national_emblem", "attributes": []},
    {"name":"remove", "attributes": []},
]

In [13]:
from test_api import CLI, CVAT_API_V1, ResourceType

class Args:
    
    def __init__(self, action="create"):
        
        self.action = action
        self.auth = ("", "")  # type tuple
        self.server_host = ""
        self.server_port = 8080
        self.https = False
        self.loglevel = 20

        self.name = "test"
        self.labels = [{"name":"text", "attributes": []}]  # type list
        self.overlap = 0
        self.segment_size = 0
        self.bug = ""
        self.resource_type = ResourceType(0)
        self.resources = ["/home/phan.huy.hoang/Pictures/test_img_cvat.zip"]
        self.annotation_path = "/home/phan.huy.hoang/cvat/utils/cli/instances_default.json"
        self.annotation_format = "COCO 1.0"

        # for git only
        self.completion_verification_period = 10
        self.dataset_repository_url = ""
        self.lfs = False

args = Args()

actions = {
    'create': CLI.tasks_create,
    'delete': CLI.tasks_delete,
    'ls': CLI.tasks_list,
    'frames': CLI.tasks_frame,
    'dump': CLI.tasks_dump,
    'upload': CLI.tasks_upload
}

In [ ]:
zip_fps = sorted(glob.glob(os.path.join(save_image_fd, "*.zip")))
for i, (zip_fp, _) in tqdm(enumerate(zip(zip_fps, zip_fps)), total=len(zip_fps)):
    args.annotation_path = ""
    args.resources = [zip_fp]
    
    if "_front.zip" in zip_fp:
        mode = "front"
    elif "_back.zip" in zip_fp:
        mode = "back"
    else:
        raise NotImplementedError()

    task_name = "[eKYC_CROPPER]{}_{}".format(int(time.time()), mode)
    args.name = task_name

    with requests.Session() as session:
        api = CVAT_API_V1('{}:{}'.format(args.server_host, args.server_port), args.https)
        cli = CLI(session, api, args.auth)

        args.action = "create"
#         task_json, resp_json = actions["create"](cli, **args.__dict__)
        res = actions["create"](cli, **args.__dict__)
        gc.collect()
    
    time.sleep(60)

### Thêm attributes trong object để tiến hành annotate cho OCR

In [14]:
CVAT_URL = "http://{}:{}/".format(args.server_host, args.server_port)
API_URL = CVAT_URL + "api/v1/"
username, password = args.auth
username, password

def get_cookies():
    login_json = {"username": username, "password": password}
    login_req = requests.post(API_URL + 'auth/login', json=login_json)
    cookies = login_req.cookies
    return cookies

cookies = get_cookies()

In [123]:
JSON_UPDATE = {
  "labels": [
    {
      "id": "",  # TODO
      "name": "",  # TODO
      "attributes": [
        {
          "name": "type",
          "mutable": False,
          "input_type": "radio",
          "default_value": "none",
          "values": [
            "cmnd",
            "cccd",
            "cccd_chip",
            "none"
            ]
        }
      ],
      "deleted": False
    }
  ]
}

REQ_URL = "http://{}:{}/api/v1/tasks/{}"  # task_id

1043 --> 1062

In [126]:
task_ids = range(1043, 1063)
# task_ids = [1044]

In [127]:
for task_id in tqdm_notebook(task_ids):
    req = requests.get(req_url.format(task_id), cookies=cookies)
    assert req.status_code == 200
    
    for element in req.json()["labels"]:
        if element["name"] == "top_left":
            update_element = copy.deepcopy(element)
            break
            
    json_update = copy.deepcopy(JSON_UPDATE)
    json_update["labels"][0]["id"] = update_element["id"]
    json_update["labels"][0]["name"] = update_element["name"]

#     headers = {'X-CSRFToken': cookies['csrftoken']}
    patch_req = requests.patch(
        req_url.format(args.server_host, args.server_port, task_id),
        json=json_update,
        auth=(username, password),
    #     OR
    #     cookies=cookies,
    #     headers=headers
    )
    print("{}: {}".format(task_id, patch_req.status_code))
    time.sleep(3)

'1043: 200'
'1044: 200'
'1045: 200'
'1046: 200'
'1047: 200'
'1048: 200'
'1049: 200'
'1050: 200'
'1051: 200'
'1052: 200'
'1053: 200'
'1054: 200'
'1055: 200'
'1056: 200'
'1057: 200'
'1058: 200'
'1059: 200'
'1060: 200'
'1061: 200'
'1062: 200'
